In [1]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate


Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: datasets[audio]
Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
os.environ['PATH'] += os.pathsep + '/Users/gse/PycharmProjects/ffmpeg-2025-07-12-git-35a6de137a-full_build/bin'

In [16]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps = True,
    chunk_length_s = 10,
    stride_length_s = 2,
)

#dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"]
sample = "./audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)
print(result)

#print(result["text"])


Device set to use cpu


ValueError: ffmpeg was not found but is required to load audio files from filename

In [ ]:
start_end_text = []

for chunk in result['chunk']:
    start = chunk['timestamp'][0]
    end = chunk['timestamp'][1]
    text = chunk['text']
    start_end_text.append((start, end, text))

import pandas as pd
df = pd.DataFrame(start_end_text, columns=['start', 'end', 'text'])
df.to_csv("lsy_audio_2023_58.csv", index=False,sep="|")
display(df)
